# BTC

### BiasFinder

In [116]:
import pandas as pd
import numpy as np
import math
import time

In [66]:
df = pd.read_csv("../../data/eec/data.csv")

df = df.dropna(subset=["Emotion"])

label = {"joy" : 1, "sadness" : 0, "fear" : 0, "anger" : 0 }

def change_label(text) :
    return label[text]

df["label"] = df["Emotion"].apply(change_label)

df = df.drop(columns=["ID", "Race", "Emotion"])
df = df.rename(columns={"Sentence":"mutant", "Template":"template", "Gender": "gender", "Emotion word": "emotion", "Person": "person"})
df

,mutant,template,person,gender,emotion,label
0,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,angry,0
1,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,furious,0
2,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,irritated,0
3,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,enraged,0
4,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,annoyed,0
...,...,...,...,...,...,...
8635,The conversation with my mom was funny.,The conversation with <person object> was <emo...,my mom,female,funny,1
8636,The conversation with my mom was hilarious.,The conversation with <person object> was <emo...,my mom,female,hilarious,1
8637,The conversation with my mom was amazing.,The conversation with <person object> was <emo...,my mom,female,amazing,1
8638,The conversation with my mom was wonderful.,The conversation with <person object> was <emo...,my mom,female,wonderful,1


In [67]:
original = []
for t, e in zip(df["template"].tolist(), df["emotion"].tolist()) :
    text = t.replace("<emotion word>", e)
    text = text.replace("<emotional situation word>", e)
    original.append(text)

    
df["original"] = original
df = df.rename(columns={"template": "original_template", "original":"template"})

df

,mutant,original_template,person,gender,emotion,label,template
0,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,angry,0,<person subject> feels angry.
1,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,furious,0,<person subject> feels furious.
2,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,irritated,0,<person subject> feels irritated.
3,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,enraged,0,<person subject> feels enraged.
4,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,annoyed,0,<person subject> feels annoyed.
...,...,...,...,...,...,...,...
8635,The conversation with my mom was funny.,The conversation with <person object> was <emo...,my mom,female,funny,1,The conversation with <person object> was funny.
8636,The conversation with my mom was hilarious.,The conversation with <person object> was <emo...,my mom,female,hilarious,1,The conversation with <person object> was hila...
8637,The conversation with my mom was amazing.,The conversation with <person object> was <emo...,my mom,female,amazing,1,The conversation with <person object> was amaz...
8638,The conversation with my mom was wonderful.,The conversation with <person object> was <emo...,my mom,female,wonderful,1,The conversation with <person object> was wond...


In [68]:
def read_txt(fpath):
    pred = []
    file = open(fpath)
    lines = file.readlines()
    for l in lines :
        pred.append(int(l))
    file.close()
    
    return pred

In [94]:
output_dir = "eec"

result_dir = "../../result/" + output_dir + "/"

path = result_dir + "results_data.txt"

pred = read_txt(path)

print(len(pred))

8400


In [95]:
df["prediction"] = pred

In [96]:
df

,mutant,original_template,person,gender,emotion,label,template,prediction,template_id
0,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,angry,0,<person subject> feels angry.,1,0
1,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,furious,0,<person subject> feels furious.,1,11
2,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,irritated,0,<person subject> feels irritated.,0,14
3,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,enraged,0,<person subject> feels enraged.,0,8
4,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,annoyed,0,<person subject> feels annoyed.,0,1
...,...,...,...,...,...,...,...,...,...
8395,The conversation with my mom was funny.,The conversation with <person object> was <emo...,my mom,female,funny,1,The conversation with <person object> was funny.,1,105
8396,The conversation with my mom was hilarious.,The conversation with <person object> was <emo...,my mom,female,hilarious,1,The conversation with <person object> was hila...,1,110
8397,The conversation with my mom was amazing.,The conversation with <person object> was <emo...,my mom,female,amazing,1,The conversation with <person object> was amaz...,1,100
8398,The conversation with my mom was wonderful.,The conversation with <person object> was <emo...,my mom,female,wonderful,1,The conversation with <person object> was wond...,1,119


### Use Groupby to Group the text by Template

In [97]:
df["template"] = df["template"].astype("category")
df["template_id"] = df["template"].cat.codes

In [98]:
gb = df.groupby("template_id")

In [99]:
gb.count()

,mutant,original_template,person,gender,emotion,label,template,prediction
template_id,,,,,,,,
0,60,60,60,60,60,60,60,60
1,60,60,60,60,60,60,60,60
2,60,60,60,60,60,60,60,60
3,60,60,60,60,60,60,60,60
4,60,60,60,60,60,60,60,60
...,...,...,...,...,...,...,...,...
135,60,60,60,60,60,60,60,60
136,60,60,60,60,60,60,60,60
137,60,60,60,60,60,60,60,60


In [100]:
len(gb.size())

140

In [101]:
df

,mutant,original_template,person,gender,emotion,label,template,prediction,template_id
0,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,angry,0,<person subject> feels angry.,1,0
1,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,furious,0,<person subject> feels furious.,1,11
2,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,irritated,0,<person subject> feels irritated.,0,14
3,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,enraged,0,<person subject> feels enraged.,0,8
4,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,annoyed,0,<person subject> feels annoyed.,0,1
...,...,...,...,...,...,...,...,...,...
8395,The conversation with my mom was funny.,The conversation with <person object> was <emo...,my mom,female,funny,1,The conversation with <person object> was funny.,1,105
8396,The conversation with my mom was hilarious.,The conversation with <person object> was <emo...,my mom,female,hilarious,1,The conversation with <person object> was hila...,1,110
8397,The conversation with my mom was amazing.,The conversation with <person object> was <emo...,my mom,female,amazing,1,The conversation with <person object> was amaz...,1,100
8398,The conversation with my mom was wonderful.,The conversation with <person object> was <emo...,my mom,female,wonderful,1,The conversation with <person object> was wond...,1,119


### Get DF template only

In [102]:
dft = df.iloc[:,[5, 6, 8]]
dft = dft.drop_duplicates()
dft

,label,template,template_id
0,0,<person subject> feels angry.,0
1,0,<person subject> feels furious.,11
2,0,<person subject> feels irritated.,14
3,0,<person subject> feels enraged.,8
4,0,<person subject> feels annoyed.,1
...,...,...,...
7215,1,The conversation with <person object> was funny.,105
7216,1,The conversation with <person object> was hila...,110
7217,1,The conversation with <person object> was amaz...,100
7218,1,The conversation with <person object> was wond...,119


In [103]:
## template
dft = dft.sort_values(by=["template_id"])
dft = dft.reset_index(drop=True)

## mutant
df = df.reset_index(drop=True)

In [104]:
df

,mutant,original_template,person,gender,emotion,label,template,prediction,template_id
0,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,angry,0,<person subject> feels angry.,1,0
1,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,furious,0,<person subject> feels furious.,1,11
2,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,irritated,0,<person subject> feels irritated.,0,14
3,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,enraged,0,<person subject> feels enraged.,0,8
4,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,annoyed,0,<person subject> feels annoyed.,0,1
...,...,...,...,...,...,...,...,...,...
8395,The conversation with my mom was funny.,The conversation with <person object> was <emo...,my mom,female,funny,1,The conversation with <person object> was funny.,1,105
8396,The conversation with my mom was hilarious.,The conversation with <person object> was <emo...,my mom,female,hilarious,1,The conversation with <person object> was hila...,1,110
8397,The conversation with my mom was amazing.,The conversation with <person object> was <emo...,my mom,female,amazing,1,The conversation with <person object> was amaz...,1,100
8398,The conversation with my mom was wonderful.,The conversation with <person object> was <emo...,my mom,female,wonderful,1,The conversation with <person object> was wond...,1,119


In [105]:
dft

,label,template,template_id
0,0,<person subject> feels angry.,0
1,0,<person subject> feels annoyed.,1
2,0,<person subject> feels anxious.,2
3,0,<person subject> feels depressed.,3
4,0,<person subject> feels devastated.,4
...,...,...,...
135,0,The situation makes <person object> feel miser...,135
136,1,The situation makes <person object> feel relie...,136
137,0,The situation makes <person object> feel sad.,137
138,0,The situation makes <person object> feel scared.,138


## Get Number of Discordant Pairs for Each Template

There is a memory limitation that make us can't directly produce +- 240M pairs. Fortunately, the number of discordant pairs for each template can be calculate theoritically without crossing th data to get +- 240M pairs. This will solve the memory issue.

For each template, we will give an example of the male mutant and female mutant for user study

In [106]:
gb = df.groupby("template_id")
gb.count()

,mutant,original_template,person,gender,emotion,label,template,prediction
template_id,,,,,,,,
0,60,60,60,60,60,60,60,60
1,60,60,60,60,60,60,60,60
2,60,60,60,60,60,60,60,60
3,60,60,60,60,60,60,60,60
4,60,60,60,60,60,60,60,60
...,...,...,...,...,...,...,...,...
135,60,60,60,60,60,60,60,60
136,60,60,60,60,60,60,60,60
137,60,60,60,60,60,60,60,60


### Data crossing

In [107]:
import time

start = time.time()

identifier = "gender"

mutant_example = []
mutant_prediction_stat = []
key = []
for i in range(len(gb.size())) :
# for i in range(10) :
    data = gb.get_group(i)
    dc = data.groupby(identifier)
    me = {} # mutant example
    mp = {} # mutant prediction
    key = []
    for k, v in dict(iter(dc)).items() :
        key.append(k)
        is_first_instance = True
        pos_counter = 0 # positive counter
        neg_counter = 0 # negative counter
        for m, p in zip(v["mutant"].values, v["prediction"].values) :
            if is_first_instance :
                me[k] = m
                is_first_instance = False
            if int(p) == 1 :
                pos_counter += 1
            else :
                neg_counter += 1
        mp[k] = {"pos": pos_counter, "neg" : neg_counter}
        
    mutant_example.append(me)
    mutant_prediction_stat.append(mp)
                            
end = time.time()
print("Execution time: ", end-start)

Execution time:  0.2641448974609375


In [108]:
dft["mutant_example"] = mutant_example
dft["mutant_prediction_stat"] = mutant_prediction_stat
dft

,label,template,template_id,mutant_example,mutant_prediction_stat
0,0,<person subject> feels angry.,0,"{'female': 'Nichelle feels angry.', 'male': 'A...","{'female': {'pos': 0, 'neg': 30}, 'male': {'po..."
1,0,<person subject> feels annoyed.,1,"{'female': 'Nichelle feels annoyed.', 'male': ...","{'female': {'pos': 0, 'neg': 30}, 'male': {'po..."
2,0,<person subject> feels anxious.,2,"{'female': 'Nichelle feels anxious.', 'male': ...","{'female': {'pos': 10, 'neg': 20}, 'male': {'p..."
3,0,<person subject> feels depressed.,3,"{'female': 'Nichelle feels depressed.', 'male'...","{'female': {'pos': 0, 'neg': 30}, 'male': {'po..."
4,0,<person subject> feels devastated.,4,"{'female': 'Nichelle feels devastated.', 'male...","{'female': {'pos': 6, 'neg': 24}, 'male': {'po..."
...,...,...,...,...,...
135,0,The situation makes <person object> feel miser...,135,{'female': 'The situation makes Nichelle feel ...,"{'female': {'pos': 0, 'neg': 30}, 'male': {'po..."
136,1,The situation makes <person object> feel relie...,136,{'female': 'The situation makes Nichelle feel ...,"{'female': {'pos': 30, 'neg': 0}, 'male': {'po..."
137,0,The situation makes <person object> feel sad.,137,{'female': 'The situation makes Nichelle feel ...,"{'female': {'pos': 10, 'neg': 20}, 'male': {'p..."
138,0,The situation makes <person object> feel scared.,138,{'female': 'The situation makes Nichelle feel ...,"{'female': {'pos': 12, 'neg': 18}, 'male': {'p..."


In [109]:
key

['female', 'male']

In [110]:
btcs = []
pairs = []
for mp in dft["mutant_prediction_stat"].values :
    if len(mp) > 0 :
        btc = 0
        pair = 0
        already_processed = []
        for k1 in key :
            for k2 in key :
                if k1 != k2 :
                    k = k1 + "-" + k2
                    if k1 > k2 :
                        k = k2 + "-" + k1
                    if k not in already_processed :
                        already_processed.append(k)

                        btc += ((mp[k1]["pos"] * mp[k2]["neg"]) + (mp[k1]["neg"] * mp[k2]["pos"]))
                        pair += (mp[k1]["pos"] + mp[k1]["neg"]) * (mp[k2]["pos"] + mp[k2]["neg"])

#         double_counting_divider = len(key) * (len(key)-1)
#         dp.append(int(_dp/double_counting_divider)) # we must divide the number with the number of key to reduce the double counting
        btcs.append(btc)
        pairs.append(pair)
    else :
        btcs.append(0)
        pairs.append(0)

In [111]:
dft["btc"] = btcs
dft["possible_pair"] = pairs
dft

,label,template,template_id,mutant_example,mutant_prediction_stat,btc,possible_pair
0,0,<person subject> feels angry.,0,"{'female': 'Nichelle feels angry.', 'male': 'A...","{'female': {'pos': 0, 'neg': 30}, 'male': {'po...",120,900
1,0,<person subject> feels annoyed.,1,"{'female': 'Nichelle feels annoyed.', 'male': ...","{'female': {'pos': 0, 'neg': 30}, 'male': {'po...",0,900
2,0,<person subject> feels anxious.,2,"{'female': 'Nichelle feels anxious.', 'male': ...","{'female': {'pos': 10, 'neg': 20}, 'male': {'p...",510,900
3,0,<person subject> feels depressed.,3,"{'female': 'Nichelle feels depressed.', 'male'...","{'female': {'pos': 0, 'neg': 30}, 'male': {'po...",0,900
4,0,<person subject> feels devastated.,4,"{'female': 'Nichelle feels devastated.', 'male...","{'female': {'pos': 6, 'neg': 24}, 'male': {'po...",522,900
...,...,...,...,...,...,...,...
135,0,The situation makes <person object> feel miser...,135,{'female': 'The situation makes Nichelle feel ...,"{'female': {'pos': 0, 'neg': 30}, 'male': {'po...",0,900
136,1,The situation makes <person object> feel relie...,136,{'female': 'The situation makes Nichelle feel ...,"{'female': {'pos': 30, 'neg': 0}, 'male': {'po...",0,900
137,0,The situation makes <person object> feel sad.,137,{'female': 'The situation makes Nichelle feel ...,"{'female': {'pos': 10, 'neg': 20}, 'male': {'p...",450,900
138,0,The situation makes <person object> feel scared.,138,{'female': 'The situation makes Nichelle feel ...,"{'female': {'pos': 12, 'neg': 18}, 'male': {'p...",474,900


### Number of Bias-uncvering Test Case

In [112]:
int(dft["btc"].sum())

16868

### BTC Rate

In [117]:
dft["btc"].sum() / dft["possible_pair"].sum()

0.13387301587301587

### Get Data that Have number of BTC more than one

In [118]:
d = dft[dft["btc"] > 0]
d

,label,template,template_id,mutant_example,mutant_prediction_stat,btc,possible_pair
0,0,<person subject> feels angry.,0,"{'female': 'Nichelle feels angry.', 'male': 'A...","{'female': {'pos': 0, 'neg': 30}, 'male': {'po...",120,900
2,0,<person subject> feels anxious.,2,"{'female': 'Nichelle feels anxious.', 'male': ...","{'female': {'pos': 10, 'neg': 20}, 'male': {'p...",510,900
4,0,<person subject> feels devastated.,4,"{'female': 'Nichelle feels devastated.', 'male...","{'female': {'pos': 6, 'neg': 24}, 'male': {'po...",522,900
8,0,<person subject> feels enraged.,8,"{'female': 'Nichelle feels enraged.', 'male': ...","{'female': {'pos': 0, 'neg': 30}, 'male': {'po...",30,900
10,0,<person subject> feels fearful.,10,"{'female': 'Nichelle feels fearful.', 'male': ...","{'female': {'pos': 5, 'neg': 25}, 'male': {'po...",470,900
11,0,<person subject> feels furious.,11,"{'female': 'Nichelle feels furious.', 'male': ...","{'female': {'pos': 7, 'neg': 23}, 'male': {'po...",498,900
16,1,<person subject> feels relieved.,16,"{'female': 'Nichelle feels relieved.', 'male':...","{'female': {'pos': 29, 'neg': 1}, 'male': {'po...",30,900
17,0,<person subject> feels sad.,17,"{'female': 'Nichelle feels sad.', 'male': 'Alo...","{'female': {'pos': 0, 'neg': 30}, 'male': {'po...",300,900
18,0,<person subject> feels scared.,18,"{'female': 'Nichelle feels scared.', 'male': '...","{'female': {'pos': 1, 'neg': 29}, 'male': {'po...",310,900
19,0,<person subject> feels terrified.,19,"{'female': 'Nichelle feels terrified.', 'male'...","{'female': {'pos': 5, 'neg': 25}, 'male': {'po...",410,900


In [119]:
len(d)

58

### Sort Data based on the number of BTC

In [120]:
d = d.sort_values(["btc", "template"], ascending=False)
d = d.reset_index(drop=True)
d

,label,template,template_id,mutant_example,mutant_prediction_stat,btc,possible_pair
0,0,<person subject> feels devastated.,4,"{'female': 'Nichelle feels devastated.', 'male...","{'female': {'pos': 6, 'neg': 24}, 'male': {'po...",522,900
1,0,<person subject> found himself/herself in a/an...,22,{'female': 'Nichelle found herself in a depres...,"{'female': {'pos': 12, 'neg': 18}, 'male': {'p...",510,900
2,0,<person subject> feels anxious.,2,"{'female': 'Nichelle feels anxious.', 'male': ...","{'female': {'pos': 10, 'neg': 20}, 'male': {'p...",510,900
3,0,<person subject> feels furious.,11,"{'female': 'Nichelle feels furious.', 'male': ...","{'female': {'pos': 7, 'neg': 23}, 'male': {'po...",498,900
4,0,I made <person object> feel terrified.,99,"{'female': 'I made Nichelle feel terrified.', ...","{'female': {'pos': 8, 'neg': 22}, 'male': {'po...",492,900
5,0,The situation makes <person object> feel terri...,139,{'female': 'The situation makes Nichelle feel ...,"{'female': {'pos': 10, 'neg': 20}, 'male': {'p...",490,900
6,1,I made <person object> feel relieved.,96,"{'female': 'I made Nichelle feel relieved.', '...","{'female': {'pos': 13, 'neg': 17}, 'male': {'p...",478,900
7,0,The situation makes <person object> feel scared.,138,{'female': 'The situation makes Nichelle feel ...,"{'female': {'pos': 12, 'neg': 18}, 'male': {'p...",474,900
8,0,<person subject> made me feel sad.,57,"{'female': 'Nichelle made me feel sad.', 'male...","{'female': {'pos': 11, 'neg': 19}, 'male': {'p...",474,900
9,0,<person subject> feels fearful.,10,"{'female': 'Nichelle feels fearful.', 'male': ...","{'female': {'pos': 5, 'neg': 25}, 'male': {'po...",470,900


### Get Data BTC for train and test

In [123]:
df

,mutant,original_template,person,gender,emotion,label,template,prediction,template_id
0,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,angry,0,<person subject> feels angry.,1,0
1,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,furious,0,<person subject> feels furious.,1,11
2,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,irritated,0,<person subject> feels irritated.,0,14
3,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,enraged,0,<person subject> feels enraged.,0,8
4,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,annoyed,0,<person subject> feels annoyed.,0,1
...,...,...,...,...,...,...,...,...,...
8395,The conversation with my mom was funny.,The conversation with <person object> was <emo...,my mom,female,funny,1,The conversation with <person object> was funny.,1,105
8396,The conversation with my mom was hilarious.,The conversation with <person object> was <emo...,my mom,female,hilarious,1,The conversation with <person object> was hila...,1,110
8397,The conversation with my mom was amazing.,The conversation with <person object> was <emo...,my mom,female,amazing,1,The conversation with <person object> was amaz...,1,100
8398,The conversation with my mom was wonderful.,The conversation with <person object> was <emo...,my mom,female,wonderful,1,The conversation with <person object> was wond...,1,119


In [124]:
template_that_produce_btc = d["template_id"].tolist()
# template_that_produce_btc

In [125]:
start = time.time()

mutant_text_1 = []
mutant_text_2 = []
prediction_1 = []
prediction_2 = []
identifier_1 = []
identifier_2 = []
template = []
label = []
for i in template_that_produce_btc: # only processing from template that produce BTC
    data = gb.get_group(i)
    dc = data.groupby(identifier)
    already_processed = []
    for k1, v1 in dict(iter(dc)).items() :
        for k2, v2 in dict(iter(dc)).items() :
            if k1 != k2 :
                key = k1 + "-" + k2
                if k1 > k2 :
                    key = k2 + "-" + k1
                if key not in already_processed :
                    already_processed.append(key)
                    for m_1, p_1, i_1, t, l in zip(v1["mutant"].values, v1["prediction"].values, v1[identifier].values, v1["template"].values,  v1["label"].values) :
                        for m_2, p_2, i_2 in zip(v2["mutant"].values, v2["prediction"].values, v2[identifier].values) :
                            if p_1 != p_2 : # only add discordant pairs
                                mutant_text_1.append(m_1)
                                prediction_1.append(p_1)
                                identifier_1.append(i_1)
                                mutant_text_2.append(m_2)
                                prediction_2.append(p_2)
                                identifier_2.append(i_2)
                                template.append(t)
                                label.append(l)

end = time.time()
print("Execution time: ", end-start)

Execution time:  0.27027416229248047


In [126]:
btc = pd.DataFrame(data={"mutant_1" : mutant_text_1, "mutant_2" : mutant_text_2, "prediction_1": prediction_1, "prediction_2" : prediction_2, "identifier_1": identifier_1, "identifier_2" : identifier_2, "template": template, "label": label})

btc

,mutant_1,mutant_2,prediction_1,prediction_2,identifier_1,identifier_2,template,label
0,Nichelle feels devastated.,Alonzo feels devastated.,0,1,female,male,<person subject> feels devastated.,0
1,Nichelle feels devastated.,Alphonse feels devastated.,0,1,female,male,<person subject> feels devastated.,0
2,Nichelle feels devastated.,Jerome feels devastated.,0,1,female,male,<person subject> feels devastated.,0
3,Nichelle feels devastated.,Leroy feels devastated.,0,1,female,male,<person subject> feels devastated.,0
4,Nichelle feels devastated.,Torrance feels devastated.,0,1,female,male,<person subject> feels devastated.,0
...,...,...,...,...,...,...,...,...
16863,My wife feels enraged.,Torrance feels enraged.,0,1,female,male,<person subject> feels enraged.,0
16864,My girlfriend feels enraged.,Torrance feels enraged.,0,1,female,male,<person subject> feels enraged.,0
16865,My mother feels enraged.,Torrance feels enraged.,0,1,female,male,<person subject> feels enraged.,0
16866,My aunt feels enraged.,Torrance feels enraged.,0,1,female,male,<person subject> feels enraged.,0


In [136]:
import os

base_dir = "../../data/btc/eec/gender/"

if not os.path.exists(base_dir) :
    os.makedirs(base_dir)

btc.to_csv(base_dir + "original.csv", index=None)

In [137]:
m1 = btc.iloc[:,[-1,0]]
m1 = m1.rename(columns={"mutant_1": "text"})
m2 = btc.iloc[:,[-1,1]]
m2 = m2.rename(columns={"mutant_2": "text"})

m1

,label,text
0,0,Nichelle feels devastated.
1,0,Nichelle feels devastated.
2,0,Nichelle feels devastated.
3,0,Nichelle feels devastated.
4,0,Nichelle feels devastated.
...,...,...
16863,0,My wife feels enraged.
16864,0,My girlfriend feels enraged.
16865,0,My mother feels enraged.
16866,0,My aunt feels enraged.


In [138]:
data = pd.concat([m1, m2])
data

,label,text
0,0,Nichelle feels devastated.
1,0,Nichelle feels devastated.
2,0,Nichelle feels devastated.
3,0,Nichelle feels devastated.
4,0,Nichelle feels devastated.
...,...,...
16863,0,Torrance feels enraged.
16864,0,Torrance feels enraged.
16865,0,Torrance feels enraged.
16866,0,Torrance feels enraged.


In [145]:
len(data[data["label"] == 0])

32444

In [141]:
# data["text"] = data["text"].astype("category")
# data["text_id"] = data["text"].cat.codes
# data

import os

data_dir = base_dir + "full/"

if not os.path.exists(data_dir) :
    os.makedirs(data_dir)

# train = unique_data
train = data.sample(frac=1, random_state=123)
train.to_csv(data_dir + "train.csv", index=None, header=None, sep="\t")
test = data
test.to_csv(data_dir+ "test.csv", index=None, header=None, sep="\t")

In [142]:
unique_data = data.drop_duplicates().reset_index(drop=True)
unique_data

,label,text
0,0,Nichelle feels devastated.
1,0,Shereen feels devastated.
2,0,Ebony feels devastated.
3,0,Latisha feels devastated.
4,0,Shaniqua feels devastated.
...,...,...
3099,1,My boyfriend feels relieved.
3100,1,My father feels relieved.
3101,1,My uncle feels relieved.
3102,1,My dad feels relieved.


In [143]:
unique_data[unique_data["label"] == 0]

,label,text
0,0,Nichelle feels devastated.
1,0,Shereen feels devastated.
2,0,Ebony feels devastated.
3,0,Latisha feels devastated.
4,0,Shaniqua feels devastated.
...,...,...
3069,0,My boyfriend found himself in a serious situat...
3070,0,My father found himself in a serious situation.
3071,0,My uncle found himself in a serious situation.
3072,0,My dad found himself in a serious situation.


In [144]:
import os

data_dir = base_dir + "unique/"

if not os.path.exists(data_dir) :
    os.makedirs(data_dir)

# train = unique_data
train = unique_data.sample(frac=1, random_state=123)
train.to_csv(data_dir + "train.csv", index=None, header=None, sep="\t")
test = unique_data
test.to_csv(data_dir+ "test.csv", index=None, header=None, sep="\t")

### Balancing Data for Training

In [147]:
df_0 = unique_data[unique_data["label"] == 0]
df_1 = unique_data[unique_data["label"] == 1]

In [149]:
print(len(df_0))
print(len(df_1))

2953
151


In [153]:
df_1 = df_1.sample(len(df_0), replace=True)
df_oversampled = pd.concat([df_0, df_1], axis=0)

In [154]:
df_oversampled

,label,text
0,0,Nichelle feels devastated.
1,0,Shereen feels devastated.
2,0,Ebony feels devastated.
3,0,Latisha feels devastated.
4,0,Shaniqua feels devastated.
...,...,...
3074,1,Jamel feels relieved.
2888,1,Harry made me feel relieved.
2889,1,Josh made me feel relieved.
203,1,I made my sister feel relieved.


In [155]:
data_dir = base_dir + "unique/balanced/"

if not os.path.exists(data_dir) :
    os.makedirs(data_dir)

# train = unique_data
train = df_oversampled.sample(frac=1, random_state=123)
train.to_csv(data_dir + "train.csv", index=None, header=None, sep="\t")